In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [ ]:
bac = pd.read_feather('../input/bac_market_data.feather')
bac = bac.bfill()

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
bac[['volume', 'close', 'open',
     'returnsClosePrevRaw1', 'returnsOpenPrevRaw1',
     'returnsClosePrevMktres1', 'returnsOpenPrevMktres1',
     'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',
     'returnsClosePrevMktres10', 'returnsOpenPrevMktres10',
    'returnsOpenNextMktres10']] = scaler.fit_transform(bac[['volume', 'close', 'open',
                                                            'returnsClosePrevRaw1', 'returnsOpenPrevRaw1',
                                                            'returnsClosePrevMktres1', 'returnsOpenPrevMktres1',
                                                            'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',
                                                            'returnsClosePrevMktres10', 'returnsOpenPrevMktres10',
                                                            'returnsOpenNextMktres10']])

In [ ]:
trn = bac[bac['time'].dt.year<2016]
tst = bac[bac['time'].dt.year==2016]

In [ ]:
trn_X = trn[['volume', 'close', 'open',
       'returnsClosePrevRaw1', 'returnsOpenPrevRaw1',
       'returnsClosePrevMktres1', 'returnsOpenPrevMktres1',
       'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',
       'returnsClosePrevMktres10', 'returnsOpenPrevMktres10',
       'returnsOpenNextMktres10']]
trn_y = trn['y']

In [ ]:
tst_X = tst[['volume', 'close', 'open',
       'returnsClosePrevRaw1', 'returnsOpenPrevRaw1',
       'returnsClosePrevMktres1', 'returnsOpenPrevMktres1',
       'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',
       'returnsClosePrevMktres10', 'returnsOpenPrevMktres10',
       'returnsOpenNextMktres10']]
tst_y = tst['y']

In [ ]:
from keras.layers import Input, Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Model
import time #helper libraries

In [ ]:
def df_to_3d(df):
    vals = df.values
    return vals.reshape(vals.shape[0], 1, vals.shape[1])

In [ ]:
inputs = Input(shape=(1, trn_X.shape[1],))

x = LSTM(50, activation='relu', return_sequences=True)(inputs)
x = Dropout(0.2)(x)
x = LSTM(100, activation='relu')(x)
x = Dropout(0.2)(x)

predictions = Dense(1, activation='tanh')(x)

start = time.time()
model = Model(inputs=inputs, outputs=predictions)
model.compile(loss='mse', optimizer='rmsprop')
print ('compilation time : ', time.time() - start)

In [ ]:
model.fit(
    df_to_3d(trn_X), trn_y,
    batch_size=64, epochs=12,
    validation_data=(df_to_3d(tst_X), tst_y))

In [ ]:
preds = model.predict(
    df_to_3d(pd.concat([trn_X, tst_X]))
)

In [ ]:
eval_model = pd.DataFrame({'date': bac['time'].dt.date,
                           'X': preds.flatten(),
                           'y': bac['y']})

In [ ]:
def score_model(X, y):
    adj_X = np.array(X) + 1
    adj_y = np.array(y) + 1
    relative_per_diffs = []
    for val_X, val_y in zip(adj_X, adj_y):
        # calculate relative percent difference
        relative_per_diffs.append( (val_X - val_y) / (np.abs(val_X) + np.abs(val_y)) )
        #print('{:>8.3f} <-> {:>8.3f} = {:>8.3f}'.format(val_X, val_y, relative_per_diffs[-1]))
    # adjust relative percent differnces scale from [-2, 2] to [0, 1]
    # higher scores closer to 1 being more accurate
    relative_per_diffs = 1 - ((np.array(relative_per_diffs) + 1) / 2)
    return list(relative_per_diffs)

In [ ]:
eval_model['scores'] = score_model(eval_model['X'].values, eval_model['y'].values)

In [ ]:
def plot_vs_time(data_frame, column, calculation='mean', span=10):
    if calculation == 'mean':
        group_temp = data_frame.groupby('date')[column].mean().reset_index()
    if calculation == 'count':
        group_temp = data_frame.groupby('date')[column].count().reset_index()
    if calculation == 'nunique':
        group_temp = data_frame.groupby('date')[column].nunique().reset_index()
    group_temp = group_temp.ewm(span=span).mean()
    fig = plt.figure(figsize=(10,3))
    plt.plot(group_temp['date'], group_temp[column])
    plt.xlabel('Time')
    plt.ylabel(column)
    plt.ylim((0,1))
    plt.title('%s versus time' %column)

In [ ]:
plot_vs_time(eval_model, 'scores')

In [ ]:
print('Eval model score is {:.5f}'.format(eval_model['scores'].mean()))